# __Rotten Tomatoes Movie Ratings on Netflix__

## Summary:
1. Import Modules and Data
2. Data Exploration
    - 2.1 Netflix data
    - 2.2 Rotten Tomatoes data
3. Data Cleaning    
    - 3.1 Fill null values with "NA" in Netflix so that we can reserve more data and not have errors when merging
    - 3.2 Drop null values in Rotten Tomatoes as we 
    - 3.3 Drop irrelevant columns and/or rows
    - 3.4 Rename columns
    - 3.4 Create a new column to hold unique index to join on
4. Merge two dataframes to get Rotten Tomatoes movie ratings on Netflix
    - 4.1 Merge the two dataframes
    - 4.2 Data Cleaning
5. Load the master file to Postgres SQL

In [1]:
import pandas as pd
import datetime
from sqlalchemy import create_engine

## Steps 1-3 for Netflix Movies Dataset

### Step 1. Import data

In [2]:
# import netflix data
netflix = pd.read_csv("data/netflix_titles.csv")
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,title_release_year
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,Norm of the North: King Sized Adventure - 2019
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,9-Sep-16,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,Jandino: Whatever it Takes - 2016
2,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,#realityhigh - 2017
3,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada",8-Sep-17,2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",Automata - 2014
4,80164077,Movie,Fabrizio Copano: Solo pienso en mi,"Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,8-Sep-17,2017,TV-MA,60 min,Stand-Up Comedy,Fabrizio Copano takes audience participation t...,Fabrizio Copano: Solo pienso en mi - 2017


### Step 2. Data Exploration

In [3]:
# netflix data types. Date added is an object. We may need to convert it to a datetime.
netflix.dtypes

show_id                int64
type                  object
title                 object
director              object
cast                  object
country               object
date_added            object
release_year           int64
rating                object
duration              object
listed_in             object
description           object
title_release_year    object
dtype: object

In [4]:
# netflix data has 6234 rows and 12 total columns
netflix.shape

(6234, 13)

### Step 3. Data Cleaning

#### 3.1 Replace null values

In [5]:
# count how many null values in Netflix data
netflix.isnull().sum()

show_id                  0
type                     0
title                    0
director              1969
cast                   570
country                476
date_added              11
release_year             0
rating                  10
duration                 0
listed_in                0
description              0
title_release_year       0
dtype: int64

#### 3.2 Drop null values in Rotten Tomatoes

In [6]:
# replace null values with "NA"

netflix_clean = netflix.fillna("NA")
netflix_clean.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,title_release_year
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,Norm of the North: King Sized Adventure - 2019
1,80117401,Movie,Jandino: Whatever it Takes,NA,Jandino Asporaat,United Kingdom,9-Sep-16,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,Jandino: Whatever it Takes - 2016
2,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,#realityhigh - 2017
3,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada",8-Sep-17,2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",Automata - 2014
4,80164077,Movie,Fabrizio Copano: Solo pienso en mi,"Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,8-Sep-17,2017,TV-MA,60 min,Stand-Up Comedy,Fabrizio Copano takes audience participation t...,Fabrizio Copano: Solo pienso en mi - 2017


#### 3.3 Drop irrelevant rows

In [8]:
# filter out tv shows based on type column as we want to focus on movie ratings

netflix_clean = netflix_clean[netflix_clean["type"] != "TV Show"]

#### 3.4 Rename the columns

In [9]:
# rename column header 'listed_in' to be 'genre' and 'title' to be 'movie_title'

netflix_clean = netflix_clean.rename(columns = {"listed_in":"genre", "title":"movie_title"})

#### 3.5 Create a new column to hold unique index to join on

In [10]:
# fill in title_release_year column with cleaner title_release_year value & preview

netflix_clean["title_release_year"] = netflix_clean["movie_title"]+ "_" + netflix_clean["release_year"].astype(str)
netflix_clean.head()

,show_id,type,movie_title,director,cast,country,date_added,release_year,rating,duration,genre,description,title_release_year
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,Norm of the North: King Sized Adventure_2019
1,80117401,Movie,Jandino: Whatever it Takes,NA,Jandino Asporaat,United Kingdom,9-Sep-16,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,Jandino: Whatever it Takes_2016
2,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,#realityhigh_2017
3,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada",8-Sep-17,2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",Automata_2014
4,80164077,Movie,Fabrizio Copano: Solo pienso en mi,"Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,8-Sep-17,2017,TV-MA,60 min,Stand-Up Comedy,Fabrizio Copano takes audience participation t...,Fabrizio Copano: Solo pienso en mi_2017


## Steps 1-3 for Rotten Tomatoes Dataset

### Step 1. Import data

In [11]:
# read rotten tomatoes data

rotten_tomatoes = pd.read_csv("data/rotten_tomatoes_movies.csv")
rotten_tomatoes.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,poster_image_url,rating,genre,directors,writers,cast,...,studio_name,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,audience_top_critics_count,audience_fresh_critics_count,audience_rotten_critics_count
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,A teenager discovers he's the descendant of a ...,Though it may seem like just another Harry Pot...,https://resizing.flixster.com/p1veUpQ4ktsSHtRu...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,Craig Titley,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,20th Century Fox,Rotten,49,144,Spilled,53.0,254287.0,38,71,73
1,/m/0878835,Please Give,Kate has a lot on her mind. There's the ethics...,Nicole Holofcener's newest might seem slight i...,https://resizing.flixster.com/0AbudQ4KsB4BeXSB...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",...,Sony Pictures Classics,Certified Fresh,86,140,Upright,64.0,11567.0,43,121,19
2,/m/10,10,Blake Edwards' 10 stars Dudley Moore as George...,NaN,https://resizing.flixster.com/mF0dxH6UTa0FdkMs...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",...,Waner Bros.,Fresh,68,22,Spilled,53.0,14670.0,2,15,7
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),"A Puerto Rican youth is on trial for murder, a...",Sidney Lumet's feature debut is a superbly wri...,https://resizing.flixster.com/u-8xAyGaDVvROLiR...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,Criterion Collection,Certified Fresh,100,51,Upright,97.0,105000.0,6,51,0
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","This 1954 Disney version of Jules Verne's 20,0...","One of Disney's finest live-action adventures,...",https://resizing.flixster.com/FKExgYBHu07XLoil...,G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",...,Disney,Fresh,89,27,Upright,74.0,68860.0,5,24,3


### Step 2. Data Exploration

In [12]:
# rotten tomatoes data types
rotten_tomatoes.dtypes

rotten_tomatoes_link              object
movie_title                       object
movie_info                        object
critics_consensus                 object
poster_image_url                  object
rating                            object
genre                             object
directors                         object
writers                           object
cast                              object
in_theaters_date                  object
on_streaming_date                 object
runtime_in_minutes               float64
studio_name                       object
tomatometer_status                object
tomatometer_rating                 int64
tomatometer_count                  int64
audience_status                   object
audience_rating                  float64
audience_count                   float64
audience_top_critics_count         int64
audience_fresh_critics_count       int64
audience_rotten_critics_count      int64
dtype: object

In [13]:
# rotten tomatoes has 16,638 rows and 23 columns
rotten_tomatoes.shape

(16638, 23)

### Step 3. Data Cleaning

#### 3.1 Replace null values

In [14]:
# count how many null values in rotten tomatoes data

rotten_tomatoes.isnull().sum()

rotten_tomatoes_link                0
movie_title                         0
movie_info                         24
critics_consensus                8329
poster_image_url                    0
rating                              0
genre                              17
directors                         114
writers                          1349
cast                              284
in_theaters_date                  815
on_streaming_date                   2
runtime_in_minutes                155
studio_name                       416
tomatometer_status                  0
tomatometer_rating                  0
tomatometer_count                   0
audience_status                   252
audience_rating                   252
audience_count                    252
audience_top_critics_count          0
audience_fresh_critics_count        0
audience_rotten_critics_count       0
dtype: int64

#### 3.2 Fill Null Columns as "NA"

In [15]:
# fill null columns as "NA"

RT_clean = rotten_tomatoes.fillna("NA")
RT_clean.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,poster_image_url,rating,genre,directors,writers,cast,...,studio_name,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,audience_top_critics_count,audience_fresh_critics_count,audience_rotten_critics_count
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,A teenager discovers he's the descendant of a ...,Though it may seem like just another Harry Pot...,https://resizing.flixster.com/p1veUpQ4ktsSHtRu...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,Craig Titley,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,20th Century Fox,Rotten,49,144,Spilled,53,254287,38,71,73
1,/m/0878835,Please Give,Kate has a lot on her mind. There's the ethics...,Nicole Holofcener's newest might seem slight i...,https://resizing.flixster.com/0AbudQ4KsB4BeXSB...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",...,Sony Pictures Classics,Certified Fresh,86,140,Upright,64,11567,43,121,19
2,/m/10,10,Blake Edwards' 10 stars Dudley Moore as George...,NA,https://resizing.flixster.com/mF0dxH6UTa0FdkMs...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",...,Waner Bros.,Fresh,68,22,Spilled,53,14670,2,15,7
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),"A Puerto Rican youth is on trial for murder, a...",Sidney Lumet's feature debut is a superbly wri...,https://resizing.flixster.com/u-8xAyGaDVvROLiR...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,Criterion Collection,Certified Fresh,100,51,Upright,97,105000,6,51,0
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","This 1954 Disney version of Jules Verne's 20,0...","One of Disney's finest live-action adventures,...",https://resizing.flixster.com/FKExgYBHu07XLoil...,G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",...,Disney,Fresh,89,27,Upright,74,68860,5,24,3


#### 3.3 Rename the columns

In [16]:
# rename 'in_theaters_date' to 'RT_release_date'

RT_clean = RT_clean.rename(columns = {"in_theaters_date":"RT_release_date"})
RT_clean.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,poster_image_url,rating,genre,directors,writers,cast,...,studio_name,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,audience_top_critics_count,audience_fresh_critics_count,audience_rotten_critics_count
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,A teenager discovers he's the descendant of a ...,Though it may seem like just another Harry Pot...,https://resizing.flixster.com/p1veUpQ4ktsSHtRu...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,Craig Titley,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,20th Century Fox,Rotten,49,144,Spilled,53,254287,38,71,73
1,/m/0878835,Please Give,Kate has a lot on her mind. There's the ethics...,Nicole Holofcener's newest might seem slight i...,https://resizing.flixster.com/0AbudQ4KsB4BeXSB...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",...,Sony Pictures Classics,Certified Fresh,86,140,Upright,64,11567,43,121,19
2,/m/10,10,Blake Edwards' 10 stars Dudley Moore as George...,NA,https://resizing.flixster.com/mF0dxH6UTa0FdkMs...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",...,Waner Bros.,Fresh,68,22,Spilled,53,14670,2,15,7
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),"A Puerto Rican youth is on trial for murder, a...",Sidney Lumet's feature debut is a superbly wri...,https://resizing.flixster.com/u-8xAyGaDVvROLiR...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,Criterion Collection,Certified Fresh,100,51,Upright,97,105000,6,51,0
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","This 1954 Disney version of Jules Verne's 20,0...","One of Disney's finest live-action adventures,...",https://resizing.flixster.com/FKExgYBHu07XLoil...,G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",...,Disney,Fresh,89,27,Upright,74,68860,5,24,3


#### 3.4 Drop irrelevant columns

In [17]:
# keep only certain columns in RT_clean dataframe

RT_clean = RT_clean[["rotten_tomatoes_link", "movie_title", "RT_release_date", "tomatometer_status", "tomatometer_rating", "audience_rating"]]
RT_clean.head()

,rotten_tomatoes_link,movie_title,RT_release_date,tomatometer_status,tomatometer_rating,audience_rating
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,2/12/2010,Rotten,49,53
1,/m/0878835,Please Give,4/30/2010,Certified Fresh,86,64
2,/m/10,10,10/5/1979,Fresh,68,53
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),4/13/1957,Certified Fresh,100,97
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea",1/1/1954,Fresh,89,74


#### 3.5 Create a new column to hold unique index to join on

In [20]:
# concatenate movie title and release year into title_release_year column as key value

RT_clean_1 = RT_clean.copy()
RT_clean_1["title_release_year"] = RT_clean["movie_title"] + "_" + RT_clean['RT_release_date'].str[-4:]
RT_clean_1.head()

,rotten_tomatoes_link,movie_title,RT_release_date,tomatometer_status,tomatometer_rating,audience_rating,title_release_year
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,2/12/2010,Rotten,49,53,Percy Jackson & the Olympians: The Lightning T...
1,/m/0878835,Please Give,4/30/2010,Certified Fresh,86,64,Please Give_2010
2,/m/10,10,10/5/1979,Fresh,68,53,10_1979
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),4/13/1957,Certified Fresh,100,97,12 Angry Men (Twelve Angry Men)_1957
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea",1/1/1954,Fresh,89,74,"20,000 Leagues Under The Sea_1954"


### 4. Merge two dataframes to get Rotten Tomatoes movie ratings on Netflix

#### 4.1 Merge two dataframes

In [21]:
# join two dataframes together based on movie title and year

master_df = pd.merge(netflix_clean, RT_clean_1, on = "title_release_year", how = "inner")
master_df.head()

,show_id,type,movie_title_x,director,cast,country,date_added,release_year,rating,duration,genre,description,title_release_year,rotten_tomatoes_link,movie_title_y,RT_release_date,tomatometer_status,tomatometer_rating,audience_rating
0,70304990,Movie,Good People,Henrik Ruben Genz,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden",8-Sep-17,2014,R,90 min,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...,Good People_2014,/m/good_people_2014,Good People,9/26/2014,Rotten,9,22
1,70299204,Movie,Kidnapping Mr. Heineken,Daniel Alfredson,"Jim Sturgess, Sam Worthington, Ryan Kwanten, A...","Netherlands, Belgium, United Kingdom, United S...",8-Sep-17,2015,R,95 min,"Action & Adventure, Dramas, International Movies","When beer magnate Alfred ""Freddy"" Heineken is ...",Kidnapping Mr. Heineken_2015,/m/kidnapping_mr_heineken,Kidnapping Mr. Heineken,3/6/2015,Rotten,19,23
2,80057969,Movie,Love,Gaspar Noé,"Karl Glusman, Klara Kristin, Aomi Muyock, Ugo ...","France, Belgium",8-Sep-17,2015,NR,135 min,"Cult Movies, Dramas, Independent Movies",A man in an unsatisfying marriage recalls the ...,Love_2015,/m/love_2015,Love,10/30/2015,Rotten,40,38
3,70304988,Movie,Stonehearst Asylum,Brad Anderson,"Kate Beckinsale, Jim Sturgess, David Thewlis, ...",United States,8-Sep-17,2014,PG-13,113 min,"Horror Movies, Thrillers","In 1899, a young doctor arrives at an asylum f...",Stonehearst Asylum_2014,/m/eliza_graves,Stonehearst Asylum,10/24/2014,Rotten,53,49
4,80057700,Movie,The Runner,Austin Stark,"Nicolas Cage, Sarah Paulson, Connie Nielsen, W...",United States,8-Sep-17,2015,R,90 min,"Dramas, Independent Movies",A New Orleans politician finds his idealistic ...,The Runner_2015,/m/the_runner_1999,The Runner,8/7/2015,Rotten,24,22


In [22]:
# how many rows are there in master_df
master_df.shape

(792, 19)

#### 4.2 Data Cleaning

In [23]:
# drop 'movie_title_y' and 'RT_release_date'
master_df = master_df.drop(columns = ["movie_title_y","RT_release_date"])
master_df

,show_id,type,movie_title_x,director,cast,country,date_added,release_year,rating,duration,genre,description,title_release_year,rotten_tomatoes_link,tomatometer_status,tomatometer_rating,audience_rating
0,70304990,Movie,Good People,Henrik Ruben Genz,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden",8-Sep-17,2014,R,90 min,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...,Good People_2014,/m/good_people_2014,Rotten,9,22
1,70299204,Movie,Kidnapping Mr. Heineken,Daniel Alfredson,"Jim Sturgess, Sam Worthington, Ryan Kwanten, A...","Netherlands, Belgium, United Kingdom, United S...",8-Sep-17,2015,R,95 min,"Action & Adventure, Dramas, International Movies","When beer magnate Alfred ""Freddy"" Heineken is ...",Kidnapping Mr. Heineken_2015,/m/kidnapping_mr_heineken,Rotten,19,23
2,80057969,Movie,Love,Gaspar Noé,"Karl Glusman, Klara Kristin, Aomi Muyock, Ugo ...","France, Belgium",8-Sep-17,2015,NR,135 min,"Cult Movies, Dramas, Independent Movies",A man in an unsatisfying marriage recalls the ...,Love_2015,/m/love_2015,Rotten,40,38
3,70304988,Movie,Stonehearst Asylum,Brad Anderson,"Kate Beckinsale, Jim Sturgess, David Thewlis, ...",United States,8-Sep-17,2014,PG-13,113 min,"Horror Movies, Thrillers","In 1899, a young doctor arrives at an asylum f...",Stonehearst Asylum_2014,/m/eliza_graves,Rotten,53,49
4,80057700,Movie,The Runner,Austin Stark,"Nicolas Cage, Sarah Paulson, Connie Nielsen, W...",United States,8-Sep-17,2015,R,90 min,"Dramas, Independent Movies",A New Orleans politician finds his idealistic ...,The Runner_2015,/m/the_runner_1999,Rotten,24,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,70083535,Movie,Om Shanti Om,Farah Khan,"Shah Rukh Khan, Deepika Padukone, Shreyas Talp...",India,1-Apr-18,2007,TV-14,169 min,"Comedies, Dramas, International Movies",Reincarnated 30 years after being killed in a ...,Om Shanti Om_2007,/m/om_shanti_om,Fresh,76,77
788,70039405,Movie,Paheli,Amol Palekar,"Shah Rukh Khan, Rani Mukerji, Anupam Kher, Dil...",India,1-Apr-18,2005,TV-PG,134 min,"Dramas, International Movies, Romantic Movies",Traditional Indian folklore comes alive in thi...,Paheli_2005,/m/1219165-paheli,Rotten,57,66
789,60036359,Movie,Schindler's List,Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...",United States,1-Apr-18,1993,R,195 min,"Classic Movies, Dramas",Oskar Schindler becomes an unlikely humanitari...,Schindler's List_1993,/m/schindlers_list,Certified Fresh,97,97
790,80049281,Movie,Secret in Their Eyes,Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...","United States, United Kingdom, Spain, South Korea",1-Apr-18,2015,PG-13,111 min,"Dramas, Thrillers",A former FBI investigator reopens the haunting...,Secret in Their Eyes_2015,/m/secret_in_their_eyes_2015,Rotten,39,41


In [24]:
# rename "movie_title_x" column to be "movie_title"

master_df = master_df.rename(columns = {"movie_title_x":"movie_title"})
master_df

,show_id,type,movie_title,director,cast,country,date_added,release_year,rating,duration,genre,description,title_release_year,rotten_tomatoes_link,tomatometer_status,tomatometer_rating,audience_rating
0,70304990,Movie,Good People,Henrik Ruben Genz,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden",8-Sep-17,2014,R,90 min,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...,Good People_2014,/m/good_people_2014,Rotten,9,22
1,70299204,Movie,Kidnapping Mr. Heineken,Daniel Alfredson,"Jim Sturgess, Sam Worthington, Ryan Kwanten, A...","Netherlands, Belgium, United Kingdom, United S...",8-Sep-17,2015,R,95 min,"Action & Adventure, Dramas, International Movies","When beer magnate Alfred ""Freddy"" Heineken is ...",Kidnapping Mr. Heineken_2015,/m/kidnapping_mr_heineken,Rotten,19,23
2,80057969,Movie,Love,Gaspar Noé,"Karl Glusman, Klara Kristin, Aomi Muyock, Ugo ...","France, Belgium",8-Sep-17,2015,NR,135 min,"Cult Movies, Dramas, Independent Movies",A man in an unsatisfying marriage recalls the ...,Love_2015,/m/love_2015,Rotten,40,38
3,70304988,Movie,Stonehearst Asylum,Brad Anderson,"Kate Beckinsale, Jim Sturgess, David Thewlis, ...",United States,8-Sep-17,2014,PG-13,113 min,"Horror Movies, Thrillers","In 1899, a young doctor arrives at an asylum f...",Stonehearst Asylum_2014,/m/eliza_graves,Rotten,53,49
4,80057700,Movie,The Runner,Austin Stark,"Nicolas Cage, Sarah Paulson, Connie Nielsen, W...",United States,8-Sep-17,2015,R,90 min,"Dramas, Independent Movies",A New Orleans politician finds his idealistic ...,The Runner_2015,/m/the_runner_1999,Rotten,24,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,70083535,Movie,Om Shanti Om,Farah Khan,"Shah Rukh Khan, Deepika Padukone, Shreyas Talp...",India,1-Apr-18,2007,TV-14,169 min,"Comedies, Dramas, International Movies",Reincarnated 30 years after being killed in a ...,Om Shanti Om_2007,/m/om_shanti_om,Fresh,76,77
788,70039405,Movie,Paheli,Amol Palekar,"Shah Rukh Khan, Rani Mukerji, Anupam Kher, Dil...",India,1-Apr-18,2005,TV-PG,134 min,"Dramas, International Movies, Romantic Movies",Traditional Indian folklore comes alive in thi...,Paheli_2005,/m/1219165-paheli,Rotten,57,66
789,60036359,Movie,Schindler's List,Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...",United States,1-Apr-18,1993,R,195 min,"Classic Movies, Dramas",Oskar Schindler becomes an unlikely humanitari...,Schindler's List_1993,/m/schindlers_list,Certified Fresh,97,97
790,80049281,Movie,Secret in Their Eyes,Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...","United States, United Kingdom, Spain, South Korea",1-Apr-18,2015,PG-13,111 min,"Dramas, Thrillers",A former FBI investigator reopens the haunting...,Secret in Their Eyes_2015,/m/secret_in_their_eyes_2015,Rotten,39,41


### 5. Load the master file to Postgres

In [20]:
# Create database connection
# Make sure you create the database in Postgres first
connection_string = "postgres:postgres@localhost:5432/netflix_RT_ratings"
engine = create_engine(f'postgresql://{connection_string}')

In [21]:
# Use pandas to load json converted DataFrame into database
master_df.to_sql(name='netflix_RT_ratings', con=engine, if_exists = 'replace', index=True)